In [17]:
import os
import sys
import configparser
import pathlib

import ee
import requests
import rasterio as rio
import folium

In [25]:
cd ..

/home/nsteiner/devel/eelc


In [26]:
import eelc

/home/nsteiner/miniconda3/envs/pangeo/lib/python3.9/site-packages/pyproj/__init__.py:90: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [27]:
roi_folder = pathlib.Path('/media/nsteiner/data1/sen12ms/ROIs1868_summer_s1')
eelc.write_roi(roi_folder)

ERROR 1: PROJ: proj_identify: /home/nsteiner/miniconda3/share/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 2 is expected. It comes from another PROJ installation.


CRSError: Invalid projection: EPSG:None: (Internal Proj Error: proj_create: no database context specified)

In [15]:
def load_configs():
    config_parser = configparser.ConfigParser()
    try:
        config_parser.read_file(open('../config.ini', 'r'))
        assert 'gcs' in config_parser
    except:
        raise Exception('NASA Earthdata credentials not found, please run: write_earthdata_credentials.py')
    return dict(config_parser['gcs'])


In [7]:
def init_credentials():
    config = load_configs()
    service_account = config['service_account']
    credentials = ee.ServiceAccountCredentials(service_account, config['private_key'])
    ee.Initialize(credentials)
init_credentials()

In [8]:
area_of_interest = ee.Geometry.Polygon([[
        [-122.45, 37.74],
        [-122.45, 37.84],
        [-122.35, 37.84],
        [-122.35, 37.74],
        [-122.45, 37.74]
    ]])


In [11]:
# Define the center of our map.
lat, lon = 45.77, 4.855

my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map

In [12]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [22]:
area_of_interest

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -122.45,
              37.74
            ],
            [
              -122.45,
              37.84
            ],
            [
              -122.35,
              37.84
            ],
            [
              -122.35,
              37.74
            ],
            [
              -122.45,
              37.74
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [10]:
# Select a specific band and dates for land cover.
lc_img = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").clip(area_of_interest).select('discrete_classification');

# Set visualization parameters for land cover.
lc_vis_params = {
    'min': 1,'max': 17,
    'palette': ['05450a','086a10', '54a708', '78d203', '009900', 'c6b044',
                'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44',
                'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4', '1c0dff']
}

# Create a map.
lon, lat =  -122.45, 37.84
my_map = folium.Map(location=[lat, lon], zoom_start=7)

# Add the land cover to the map object.
my_map.add_ee_layer(lc_img, {}, 'Land Cover')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)


NameError: name 'area_of_interest' is not defined

In [21]:
import ee

def load_configs():
    config_parser = configparser.ConfigParser()
    try:
        config_parser.read_file(open('../config.ini', 'r'))
        assert 'gcs' in config_parser
    except:
        raise Exception('NASA Earthdata credentials not found, please run: write_earthdata_credentials.py')
    return dict(config_parser['gcs'])

# Initialize the Earth Engine API
config = load_configs()
service_account = config['service_account']
credentials = ee.ServiceAccountCredentials(service_account, config['private_key'])
ee.Initialize(credentials)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = config['private_key']

# Get the list of tasks
tasks = ee.batch.Task.list()


# Print the number of tasks
print(len(tasks))

6530


In [28]:
config = load_configs()

# Define the file name and the image format (GeoTIFF):
file_name = 'san_francisco_sentinel2'
format = 'GeoTIFF'
# Export the image to your Google Cloud Storage bucket:
task = ee.batch.Export.image.toCloudStorage(image=lc_img, description=file_name, 
                    fileNamePrefix=file_name, bucket=config['bucket_name'], 
                    scale=10, fileFormat=format)
task.start()



In [34]:
task.status()

{'state': 'COMPLETED',
 'description': 'san_francisco_sentinel2.tif',
 'creation_timestamp_ms': 1671668781603,
 'update_timestamp_ms': 1671668836663,
 'start_timestamp_ms': 1671668807251,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://console.developers.google.com/storage/browser/eelcchips/'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 1.2109812498092651,
 'id': 'EYYE7RUNDHFFE5U6ULGF4Q3K',
 'name': 'projects/earthengine-legacy/operations/EYYE7RUNDHFFE5U6ULGF4Q3K'}

In [28]:
# Import the necessary libraries
import google.auth
from google.auth.transport.requests import Request
from google.cloud import storage

# Set the bucket name
bucket_name = 'eelcchips'

# Set up the credentials
credentials, _ = google.auth.default()

# Create a client to access the Google Cloud Storage API
client = storage.Client(credentials=credentials)

# Get the bucket
bucket = client.bucket(bucket_name)

# List the files in the bucket
files = bucket.list_blobs()

# Print the file names

print(len(list(files)))

6329


In [4]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/nsteiner/devel/eelc/.private-key.json'

In [33]:
task.status()

{'state': 'RUNNING',
 'description': 'san_francisco_sentinel2.tif',
 'creation_timestamp_ms': 1671668781603,
 'update_timestamp_ms': 1671668809429,
 'start_timestamp_ms': 1671668807251,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'EYYE7RUNDHFFE5U6ULGF4Q3K',
 'name': 'projects/earthengine-legacy/operations/EYYE7RUNDHFFE5U6ULGF4Q3K'}

In [35]:
def get_bounds(geotiff):
    # Open the GeoTIFF file
    with rio.open(geotiff) as src:
        # Read the metadata of the raster dataset
        meta = src.meta

    # Get the bounds of the raster dataset
    left, bottom, right, top = src.bounds

    # Create a list of the corner coordinates of the raster dataset
    coordinates = [[left, top], [left, bottom], [right, bottom], [right, top]]

    # Create an ee.Geometry object from the list of coordinates
    geometry = ee.Geometry.Polygon(coordinates)
    return geometry

def get_poly_list(path_):
    poly_list = []
    for tif_ in path_.rglob('*.tif'):
        poly_ = get_bounds(tif_)
        if poly_ not in poly_list:
            poly_list.append(poly_)
        else:
            print(f'Duplicate: {tif_.to_posix()}')
    return poly_list


In [39]:
DEF_PATH = '/media/nsteiner/data1/sen12ms/ROIs1158_spring'
def_path = pathlib.Path(DEF_PATH)
poly_list = get_poly_list(def_path)


KeyboardInterrupt: 

Exception ignored in: 'rasterio._env.log_error'
Traceback (most recent call last):
  File "/home/nsteiner/miniconda3/envs/pangeo/lib/python3.9/logging/__init__.py", line 1497, in log
    def log(self, level, msg, *args, **kwargs):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1443, in _pydevd_bundle.pydevd_cython.SafeCallWrapper.__call__
  File "_pydevd_bundle/pydevd_cython.pyx", line 1744, in _pydevd_bundle.pydevd_cython.ThreadTracer.__call__
  File "/home/nsteiner/miniconda3/envs/pangeo/lib/python3.9/site-packages/debugpy/_vendored/pydevd/_pydev_bundle/pydev_is_thread_alive.py", line 9, in is_thread_alive
    def is_thread_alive(t):
KeyboardInterrupt: 


In [ ]:
# Create a map.
lon, lat =  -122.45, 37.84
my_map = folium.Map(location=[lat, lon], zoom_start=7)

# Add the land cover to the map object.
my_map.add_ee_layer(lc_img, {}, 'Land Cover')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)
